## Reading information from files

I use os and glob library for work with path and directories, where we put our files.

In [16]:
import os # for work with path/dir
import sqlite3
import pandas as pd
from glob import glob

In [17]:
# use glob to get all of the csv files
csv_files = glob("./data/*.csv")
tsv_files = glob("./data/*.tsv")

# check what we have 
print("Files for project: ")
for file in csv_files:
    print(file)
for file in tsv_files:
    print(file)    

Files for project: 
./data/tmdb.movies.csv
./data/imdb.title.crew.csv
./data/tn.movie_budgets.csv
./data/imdb.title.ratings.csv
./data/imdb.name.basics.csv
./data/imdb.title.principals.csv
./data/imdb.title.akas.csv
./data/bom.movie_gross.csv
./data/imdb.title.basics.csv
./data/rt.reviews.tsv
./data/rt.movie_info.tsv


In [18]:
# os library helps us with cleaning filenames from dots
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") # cleaning the filenames

In [19]:
# create a dictionary with keys as filenames (cleaned)
# values as their respective dataframes
files_dict = {}
# working with csv files
for filename in csv_files:
    # cleaning the file names
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") 
    # read file to DataFrame
    filename_df = pd.read_csv(filename, index_col=0)
     # use cleanfilename as a key and the df as its value
    files_dict[filename_cleaned] = filename_df
# repeat for tsv files
for filename in tsv_files:
    filename_cleaned = os.path.basename(filename).replace(".tsv", "").replace(".", "_") 
    filename_df = pd.read_csv(filename, index_col=0, delimiter='\t',encoding = 'unicode_escape')
    files_dict[filename_cleaned] = filename_df 

In [21]:
# check what we have in files_dict  
for key in files_dict.keys():
    print(key)

tmdb_movies
imdb_title_crew
tn_movie_budgets
imdb_title_ratings
imdb_name_basics
imdb_title_principals
imdb_title_akas
bom_movie_gross
imdb_title_basics
rt_reviews
rt_movie_info


In [22]:
# check what we have in files_dict  
for values in files_dict.values():
    print(values)

                 genre_ids      id original_language  \
0          [12, 14, 10751]   12444                en   
1      [14, 12, 16, 10751]   10191                en   
2            [12, 28, 878]   10138                en   
3          [16, 35, 10751]     862                en   
4            [28, 878, 12]   27205                en   
...                    ...     ...               ...   
26512             [27, 18]  488143                en   
26513             [18, 53]  485975                en   
26514         [14, 28, 12]  381231                en   
26515      [10751, 12, 28]  366854                en   
26516             [53, 27]  309885                en   

                                     original_title  popularity release_date  \
0      Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
1                          How to Train Your Dragon      28.734   2010-03-26   
2                                        Iron Man 2      28.515   2010-05-07   
3      

# Creat SQL database for storage all information

In [23]:
# this will create a sqlite file in your directory
# add name for sqlite file
db = 'movies_database.sqlite'
# check what we have
if os.path.isfile(db):
    os.remove(db)
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    print("We removed old database with name "+str(db)+" and create new one with the same name")
else: 
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    print("We create a new database "+str(db))

We removed old database with name movies_database.sqlite and create new one with the same name


In [24]:
# Write a function that converts the dataframe to a sqlite table
def create_sql_table_from_df(df, name, conn):
    # Use try except
    # it will try to make a table
    # if a table exists the function will execute whatever you put in the except part
    try:
        df.to_sql(name, conn)
        print(f"Created table {name}")
    
    # if the table exists it will tell you, and won't cause an error
    except Exception as e:
        print(f"could not make table {name}")
        print(e)
        
for name, table in files_dict.items():
    create_sql_table_from_df(table, name, conn)

Created table tmdb_movies
Created table imdb_title_crew
Created table tn_movie_budgets
Created table imdb_title_ratings
Created table imdb_name_basics
Created table imdb_title_principals
Created table imdb_title_akas
Created table bom_movie_gross
Created table imdb_title_basics
Created table rt_reviews
Created table rt_movie_info


In [25]:
#check tables which we have for work
for row in cur.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(row)

('tmdb_movies',)
('imdb_title_crew',)
('tn_movie_budgets',)
('imdb_title_ratings',)
('imdb_name_basics',)
('imdb_title_principals',)
('imdb_title_akas',)
('bom_movie_gross',)
('imdb_title_basics',)
('rt_reviews',)
('rt_movie_info',)


## We uploaded all the files what we have in the database for ease access and work. Now we can make research and analyze information from them.